In [1]:
### Setup ###
import os
import re 
import psutil
import math
import time
import yaml
import shutil
import zipfile
import random
import requests
import cv2
import itertools
import json
import copy
import pickle
import gzip
from IPython.display import display_html
from scipy.spatial import ConvexHull
from scipy.stats import entropy
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.colors as mcolors

import cv2
import torchvision.transforms as transforms
import folium
import mapclassify
import seaborn as sns

from torchvision.models import resnet50
from PIL import Image
from folium.features import DivIcon
from random import randint
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
from pyproj import Geod
from random import randint
from tqdm import tqdm

import folium
from folium.features import DivIcon
import mapclassify
import seaborn as sns

import branca.colormap as cm
from folium import LayerControl
from matplotlib.colors import LinearSegmentedColormap

import xml.etree.ElementTree as ET

In [ ]:
### Road segments ###
input_dir = "./../../example_outputs/step1_loader"
lines = gpd.read_file(os.path.join(input_dir, "LINE_EPSG4326.geojson"), driver="GeoJSON")
lines = lines[['link_id', 'side', 'geometry']]
print(len(lines))
lines.iloc[0:5]

# points = gpd.read_file(os.path.join(input_dir, "POINT_EPSG4326.geojson"), driver="GeoJSON")
# print(len(points))
# points.iloc[0:5]

unique_edges = lines["link_id"].unique()
print(len(unique_edges))

# m = lines.explore(
#     name="Toy road segments",
#     style_kwds={"alpha": 0.5, "weight": 7},
#     tiles="CartoDB dark_matter")
# m = points.explore(
#     m=m,
#     color="lightgreen",
#     name="Detected vehicles")
# m

10
64
5


/home/jlee4338/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
/home/jlee4338/.local/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [58]:
### retrieve detection results ###
input_path = os.path.join(".", "sign_detection_result.csv")
sign_df = pd.read_csv(input_path)
sign_df["consist_id"] = sign_df["file_id"].apply(lambda x: "__".join(x.split("__")[1:4]))
sign_df = sign_df[["consist_id", "class_nm", "confidence"]]
print(len(sign_df))
print(sign_df.iloc[0:1])
print()

input_path = os.path.join(".", "vehicle_detection_result.csv")
vehicle_df = pd.read_csv(input_path)
vehicle_df = vehicle_df.drop(["Unnamed: 0"], axis=1)
vehicle_df["consist_id"] = vehicle_df["index"].apply(lambda x: "__".join(x.split("__")[0:3]))
vehicle_df = vehicle_df[["consist_id", "dist_error", "cosine_similarity"]]
print(len(vehicle_df))
print(vehicle_df.iloc[0:1])

sign_vehicle_df = pd.concat([sign_df, vehicle_df])
sign_vehicle_df

4
                          consist_id       class_nm  confidence
0  cluster2_secondary_15_6__side1__3  prohibAnytime        0.78

1
                         consist_id  dist_error  cosine_similarity
0  cluster2_tertiary_27_7__side1__3    3.829481           0.876251


,consist_id,class_nm,confidence,dist_error,cosine_similarity
0,cluster2_secondary_15_6__side1__3,prohibAnytime,0.78,NaN,NaN
1,cluster2_secondary_15_6__side2__2,prohibAnytime,0.84,NaN,NaN
2,cluster2_secondary_57_3__side1__5,prohibAnytime,0.79,NaN,NaN
3,cluster2_secondary_57_3__side2__1,permit,0.35,NaN,NaN
0,cluster2_tertiary_27_7__side1__3,NaN,NaN,3.829481,0.876251


In [59]:
### filtering by three cutoffs ###
## sign detection-related cutoffs ##
# confidence
confid_cutoff = 0.3
sign_vehicle_df = sign_vehicle_df[
    (sign_vehicle_df["confidence"].notna() &
    (sign_vehicle_df["confidence"] > confid_cutoff)) |
    (sign_vehicle_df["confidence"].isna())]
print(f"after confidence: {len(sign_vehicle_df)}")

# class_nm
sign_vehicle_df = sign_vehicle_df[
    (sign_vehicle_df["class_nm"]=="permit") |
    (sign_vehicle_df["class_nm"].isna())]
print(f"after class_nm: {len(sign_vehicle_df)}")

## vehicle detection-related cutoffs ##
# cosine_similarity
cos_sim_cutoff = 0.6
sign_vehicle_df = sign_vehicle_df[
    (sign_vehicle_df["cosine_similarity"].notna() &
    (sign_vehicle_df["cosine_similarity"] > cos_sim_cutoff)) |
    (sign_vehicle_df["cosine_similarity"].isna())]
print(f"after cosine_similarity: {len(sign_vehicle_df)}")

# dist_error
dist_error_cutoff = 9
sign_vehicle_df = sign_vehicle_df[
    (sign_vehicle_df["dist_error"].notna() &
    (sign_vehicle_df["dist_error"] < dist_error_cutoff)) |
    (sign_vehicle_df["dist_error"].isna())]
print(f"after dist_error: {len(sign_vehicle_df)}")

after confidence: 5
after class_nm: 2
after cosine_similarity: 2
after dist_error: 2


In [60]:
sign_vehicle_df["link_id"] = sign_vehicle_df["consist_id"].apply(lambda x: "__".join(x.split("__")[0:1]))
sign_vehicle_df["side"] = sign_vehicle_df["consist_id"].apply(lambda x: "__".join(x.split("__")[1:2]))
sign_vehicle_df = sign_vehicle_df[["link_id", "side", 
                                   "class_nm", "confidence",
                                   "consist_id", "dist_error", "cosine_similarity"]]
sign_vehicle_df

,link_id,side,class_nm,confidence,consist_id,dist_error,cosine_similarity
3,cluster2_secondary_57_3,side2,permit,0.35,cluster2_secondary_57_3__side2__1,NaN,NaN
0,cluster2_tertiary_27_7,side1,NaN,NaN,cluster2_tertiary_27_7__side1__3,3.829481,0.876251


In [74]:
### integrate the detection results to sampled road segments
lines_result = pd.merge(lines, sign_vehicle_df,
                          on=["link_id", "side"], how="left")
lines_result["parking"] = lines_result["consist_id"].notna()
print(len(lines_result))
lines_result

10


,link_id,side,geometry,class_nm,confidence,consist_id,dist_error,cosine_similarity,parking
0,cluster2_secondary_15_6,side1,"LINESTRING (-84.39234 33.74752, -84.39199 33.7...",NaN,NaN,NaN,NaN,NaN,False
1,cluster2_secondary_15_6,side2,"LINESTRING (-84.39234 33.74752, -84.39199 33.7...",NaN,NaN,NaN,NaN,NaN,False
2,cluster2_secondary_31_17,side1,"LINESTRING (-84.41749 33.7616, -84.41749 33.76...",NaN,NaN,NaN,NaN,NaN,False
3,cluster2_secondary_31_17,side2,"LINESTRING (-84.41749 33.7616, -84.41749 33.76...",NaN,NaN,NaN,NaN,NaN,False
4,cluster2_secondary_40_3,side1,"LINESTRING (-84.37063 33.77585, -84.37046 33.7...",NaN,NaN,NaN,NaN,NaN,False
5,cluster2_secondary_40_3,side2,"LINESTRING (-84.37063 33.77585, -84.37046 33.7...",NaN,NaN,NaN,NaN,NaN,False
6,cluster2_secondary_57_3,side1,"LINESTRING (-84.38033 33.76417, -84.37969 33.7...",NaN,NaN,NaN,NaN,NaN,False
7,cluster2_secondary_57_3,side2,"LINESTRING (-84.38033 33.76417, -84.37969 33.7...",permit,0.35,cluster2_secondary_57_3__side2__1,NaN,NaN,True
8,cluster2_tertiary_27_7,side1,"LINESTRING (-84.34318 33.73842, -84.3436 33.73...",NaN,NaN,cluster2_tertiary_27_7__side1__3,3.829481,0.876251,True
9,cluster2_tertiary_27_7,side2,"LINESTRING (-84.34318 33.73842, -84.3436 33.73...",NaN,NaN,NaN,NaN,NaN,False


In [ ]:
### Export ###
output_path = os.path.join(".", "street_parking_result.csv")
lines_result.to_csv(output_path, index=True)

In [79]:
from matplotlib.colors import ListedColormap
cmap = ListedColormap(["#FF4C4C", "#4CB2FF"])

m = lines_result.explore(
    name="Street Parking",
    column="parking",         
    cmap=cmap,  
    style_kwds={"alpha": 1, "weight": 5},
    tiles="CartoDB dark_matter")
m.save("./result_map.html")
m